ETF Return Tracker
* May 19, 2025
* Version 6

Tutors:
* Anthropic's AI, Claude
* Google's AI, Gemini

This script:
* Analyzes ETF portfolio performance by calculating returns,
expenses, and net profit based on user inputs.
* Handles multiple ETFs and provide portfolio-level analysis.


Script features:
1. Prompts the user for the ETF name, expense ratio, number of shares, purchase price, purchase date, current price, and current date
2. Calculates days held between the dates
3. Computes the total return in dollars
4. Calculates the annualized return percentage
5. Estimates the total expense cost based on the expense ratio and time held
6. Determines the net return (return minus expenses)
7. Displays all results in a readable format

 I've refactored the script to include robust error handling for inputs and to allow analysis for a variable number of ETFs, some minor enhancements like ensuring dates are logical, and an annual dividend yield input for each ETF

In [ ]:
#!/usr/bin/env python3
"""
ETF Portfolio Analyzer

A Python script that analyzes a portfolio of ETFs by calculating returns,
yield income, expenses, and net profit for individual ETFs and the
combined portfolio.

Features:
- Handles analysis of a user-defined number of ETFs
- Prompts the user for ETF details (name, expense ratio, annual dividend yield,
  shares, prices, dates) with input validation and error handling
- Calculates days held between purchase and current/analysis date
- Computes capital appreciation in dollars
- Estimates total dividend yield income in dollars
- Calculates total gross return (capital appreciation + yield income)
- Calculates annualized total return percentage (including yield)
- Estimates total expense cost based on the expense ratio and time held
- Determines the net return (total gross return minus expenses)
- Provides portfolio-level analysis including asset allocation percentages
- Displays results in a readable format

Version: 6
Date: May 19, 2025
Author: Created with assistance from Anthropic's Claude, Refactored by Google's Gemini
"""

import datetime

def get_validated_float_input(prompt_message, non_negative=False, strictly_positive=False):
    """Gets a validated float input from the user."""
    while True:
        try:
            value = float(input(prompt_message))
            if non_negative and value < 0:
                print("Value cannot be negative. Please try again.")
                continue
            if strictly_positive and value <= 0:
                print("Value must be greater than zero. Please try again.")
                continue
            return value
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

def get_validated_date_input(prompt_message):
    """Gets a validated date input from the user in YYYY-MM-DD format."""
    while True:
        try:
            date_str = input(prompt_message)
            datetime.datetime.strptime(date_str, "%Y-%m-%d")
            return date_str
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

def calculate_days_held(start_date_str, end_date_str):
    """
    Calculate the number of days between two dates.

    Args:
        start_date_str (str): The starting date in YYYY-MM-DD format
        end_date_str (str): The ending date in YYYY-MM-DD format

    Returns:
        int: Number of days between the two dates, or 0 if dates are invalid.
    """
    try:
        start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
        if end_date < start_date:
            return 0
        days_held = (end_date - start_date).days
        return days_held
    except ValueError:
        print("Error: Invalid date format encountered in calculation.")
        return 0

def calculate_annualized_return_from_totals(initial_value, final_value_gross, days_held):
    """
    Calculate the annualized total return percentage from total values.
    Final value should be gross (i.e., before expenses, but including income like yield).

    Args:
        initial_value (float): The total initial investment value.
        final_value_gross (float): The total final value, including capital appreciation and income.
        days_held (int): Number of days the investment was held.

    Returns:
        float: Annualized total return as a percentage.
    """
    if days_held <= 0:
        return 0.0
    if initial_value <= 0: # Avoid division by zero or nonsensical returns
        return 0.0

    # Simple total return (gross)
    simple_total_return = (final_value_gross - initial_value) / initial_value
    annualized_return = (1 + simple_total_return) ** (365.0 / days_held) - 1
    return annualized_return * 100  # Convert to percentage

def get_etf_data(etf_number):
    """
    Prompt the user for ETF details with validation.

    Args:
        etf_number (int): The ETF number for display purposes

    Returns:
        dict: ETF data including name, expense ratio, yield, shares, prices, and dates
    """
    print(f"\nEnter details for ETF #{etf_number}:")
    print("-" * 30)

    etf_data = {}
    etf_data["name"] = input(f"Enter ETF #{etf_number} name (e.g., VTI, BND): ").strip()
    while not etf_data["name"]:
        print("ETF name cannot be empty.")
        etf_data["name"] = input(f"Enter ETF #{etf_number} name (e.g., VTI, BND): ").strip()

    etf_data["expense_ratio_percent"] = get_validated_float_input(
        f"Enter expense ratio for {etf_data['name']} (as percentage, e.g., 0.05 for 0.05%): ",
        non_negative=True
    )
    etf_data["annual_yield_percent"] = get_validated_float_input(
        f"Enter current annual dividend yield for {etf_data['name']} (as percentage, e.g., 1.5 for 1.5%): ",
        non_negative=True
    )
    etf_data["shares"] = get_validated_float_input(
        f"Enter number of shares for {etf_data['name']}: ",
        strictly_positive=True
    )
    etf_data["start_price"] = get_validated_float_input(
        f"Enter price per share on purchase date for {etf_data['name']} ($): ",
        strictly_positive=True
    )

    while True:
        etf_data["start_date"] = get_validated_date_input(
            f"Enter purchase date for {etf_data['name']} (YYYY-MM-DD): "
        )
        etf_data["end_date"] = get_validated_date_input(
            f"Enter current/analysis date for {etf_data['name']} (YYYY-MM-DD): "
        )
        try:
            start_dt = datetime.datetime.strptime(etf_data["start_date"], "%Y-%m-%d")
            end_dt = datetime.datetime.strptime(etf_data["end_date"], "%Y-%m-%d")
            if end_dt < start_dt:
                print("Error: End date cannot be before purchase date. Please re-enter dates.")
                continue
            break
        except ValueError:
            print("A date formatting error occurred. Please try re-entering dates.")
            continue

    etf_data["end_price"] = get_validated_float_input(
        f"Enter current/analysis price per share for {etf_data['name']} ($): ",
        non_negative=True
    )
    return etf_data

def analyze_etf(etf_data):
    """
    Analyze a single ETF based on provided data.

    Args:
        etf_data (dict): ETF data

    Returns:
        dict: Analysis results including returns, expenses, and performance metrics
    """
    results = {}
    results["name"] = etf_data["name"]
    results["annual_yield_percent_input"] = etf_data["annual_yield_percent"]


    days_held = calculate_days_held(etf_data["start_date"], etf_data["end_date"])
    results["days_held"] = days_held
    years_held = days_held / 365.0 if days_held > 0 else 0.0

    initial_investment = etf_data["shares"] * etf_data["start_price"]
    current_value = etf_data["shares"] * etf_data["end_price"]

    results["initial_investment"] = initial_investment
    results["current_value"] = current_value

    # Capital Appreciation
    capital_appreciation_dollars = current_value - initial_investment
    results["capital_appreciation_dollars"] = capital_appreciation_dollars

    # Estimated Yield Income
    # Based on initial investment and stated annual yield, prorated for holding period.
    # This is a simplification; actual dividends depend on ex-dividend dates and share price at that time.
    yield_decimal = etf_data["annual_yield_percent"] / 100.0
    estimated_annual_yield_on_initial = initial_investment * yield_decimal
    total_yield_income_dollars = estimated_annual_yield_on_initial * years_held
    results["total_yield_income_dollars"] = total_yield_income_dollars

    # Total Gross Return (Capital Appreciation + Yield Income)
    total_return_dollars_gross = capital_appreciation_dollars + total_yield_income_dollars
    results["total_return_dollars_gross"] = total_return_dollars_gross

    # Annualized Total Return (includes yield and appreciation)
    # Calculated based on gross total return before expenses
    final_value_for_annualization = initial_investment + total_return_dollars_gross
    annualized_total_return_pct = calculate_annualized_return_from_totals(
        initial_investment, final_value_for_annualization, days_held
    )
    results["annualized_total_return_pct"] = annualized_total_return_pct

    # Expenses
    expense_ratio_decimal = etf_data["expense_ratio_percent"] / 100.0
    # Expense based on initial investment, as in original script
    total_expense_dollars = initial_investment * expense_ratio_decimal * years_held
    results["total_expense_dollars"] = total_expense_dollars

    # Net Return (Total Gross Return - Expenses)
    net_return_dollars = total_return_dollars_gross - total_expense_dollars
    results["net_return_dollars"] = net_return_dollars

    return results

def display_etf_results(results):
    """
    Display analysis results for a single ETF.

    Args:
        results (dict): Analysis results for the ETF
    """
    print(f"\nResults for {results['name']}:")
    print("-" * 30)
    print(f"Days Held: {results['days_held']}")
    print(f"Initial Investment: ${results['initial_investment']:.2f}")
    print(f"Current Value (Capital Only): ${results['current_value']:.2f}")
    print(f"Capital Appreciation: ${results['capital_appreciation_dollars']:.2f}")
    print(f"Annual Dividend Yield (Input): {results['annual_yield_percent_input']:.2f}%")
    print(f"Estimated Total Yield Income: ${results['total_yield_income_dollars']:.2f}")
    print(f"Total Gross Return (Appreciation + Yield): ${results['total_return_dollars_gross']:.2f}")
    print(f"Annualized Total Return (incl. Yield): {results['annualized_total_return_pct']:.2f}%")
    print(f"Estimated Total Expense Cost: ${results['total_expense_dollars']:.2f}")
    print(f"Net Return (Gross Return - Expenses): ${results['net_return_dollars']:.2f}")

def analyze_portfolio(etf_results_list):
    """
    Analyze the combined portfolio of multiple ETFs.

    Args:
        etf_results_list (list): A list of analysis result dictionaries for each ETF.

    Returns:
        dict: Portfolio-level analysis including combined metrics and allocation data.
    """
    portfolio_summary = {
        "initial_investment": 0.0,
        "current_value": 0.0, # Sum of current capital values
        "capital_appreciation_dollars": 0.0,
        "total_yield_income_dollars": 0.0,
        "total_return_dollars_gross": 0.0,
        "total_expense_dollars": 0.0,
        "net_return_dollars": 0.0,
        "weighted_annualized_total_return_pct": 0.0,
        "allocations": []
    }

    if not etf_results_list:
        return portfolio_summary

    for res in etf_results_list:
        portfolio_summary["initial_investment"] += res["initial_investment"]
        portfolio_summary["current_value"] += res["current_value"]
        portfolio_summary["capital_appreciation_dollars"] += res["capital_appreciation_dollars"]
        portfolio_summary["total_yield_income_dollars"] += res["total_yield_income_dollars"]
        portfolio_summary["total_return_dollars_gross"] += res["total_return_dollars_gross"]
        portfolio_summary["total_expense_dollars"] += res["total_expense_dollars"]
        portfolio_summary["net_return_dollars"] += res["net_return_dollars"]

    # Calculate weighted annualized total return
    if portfolio_summary["initial_investment"] > 0:
        for res in etf_results_list:
            weight = res["initial_investment"] / portfolio_summary["initial_investment"]
            portfolio_summary["weighted_annualized_total_return_pct"] += res["annualized_total_return_pct"] * weight
    else:
        portfolio_summary["weighted_annualized_total_return_pct"] = 0.0

    # Calculate portfolio allocation percentages based on current capital value
    if portfolio_summary["current_value"] > 0:
        for res in etf_results_list:
            percentage = (res["current_value"] / portfolio_summary["current_value"]) * 100
            portfolio_summary["allocations"].append({
                "name": res["name"],
                "percentage": percentage
            })
    else:
        num_etfs = len(etf_results_list)
        default_percentage = 100.0 / num_etfs if num_etfs > 0 else 0.0
        for res in etf_results_list:
             portfolio_summary["allocations"].append({
                "name": res["name"],
                "percentage": default_percentage # Or 0.0 if initial also 0
            })
    return portfolio_summary

def display_portfolio_results(portfolio_summary):
    """
    Display analysis results for the combined portfolio.

    Args:
        portfolio_summary (dict): Portfolio-level analysis results.
    """
    print("\nCombined Portfolio Results:")
    print("=" * 40)
    print(f"Total Initial Investment: ${portfolio_summary['initial_investment']:.2f}")
    print(f"Total Current Value (Capital Only): ${portfolio_summary['current_value']:.2f}")
    print("Portfolio Allocation (based on Current Capital Value):")
    if portfolio_summary['allocations']:
        for alloc in portfolio_summary['allocations']:
            print(f"  - {alloc['name']}: {alloc['percentage']:.1f}%")
    else:
        print("  N/A (no ETFs or zero current value)")

    print(f"Total Capital Appreciation: ${portfolio_summary['capital_appreciation_dollars']:.2f}")
    print(f"Total Estimated Yield Income: ${portfolio_summary['total_yield_income_dollars']:.2f}")
    print(f"Total Gross Return (Appreciation + Yield): ${portfolio_summary['total_return_dollars_gross']:.2f}")
    print(f"Weighted Annualized Total Return (incl. Yield): {portfolio_summary['weighted_annualized_total_return_pct']:.2f}%")
    print(f"Total Estimated Expense Cost: ${portfolio_summary['total_expense_dollars']:.2f}")
    print(f"Total Net Return (Gross Return - Expenses): ${portfolio_summary['net_return_dollars']:.2f}")

def main():
    """
    Main function to run the ETF Portfolio Analyzer.
    """
    print("ETF Portfolio Analyzer")
    print("======================")

    while True:
        try:
            num_etfs = int(input("Enter the number of ETFs to analyze: "))
            if num_etfs <= 0:
                print("Please enter a positive number of ETFs.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a whole number.")

    all_etf_data = []
    for i in range(num_etfs):
        all_etf_data.append(get_etf_data(i + 1))

    all_etf_results = []
    for etf_data_item in all_etf_data: # Renamed to avoid conflict
        all_etf_results.append(analyze_etf(etf_data_item))

    print("\n--- Individual ETF Analysis ---")
    for results in all_etf_results:
        display_etf_results(results)

    if all_etf_results:
        print("\n--- Combined Portfolio Analysis ---")
        portfolio_summary = analyze_portfolio(all_etf_results)
        display_portfolio_results(portfolio_summary)
    else:
        print("\nNo ETF data to analyze for portfolio summary.")

    print("\nAnalysis complete.")

if __name__ == "__main__":
    main()

ETF Portfolio Analyzer
Enter the number of ETFs to analyze: 2

Enter details for ETF #1:
------------------------------
Enter ETF #1 name (e.g., VTI, BND): VTI
Enter expense ratio for VTI (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter current annual dividend yield for VTI (as percentage, e.g., 1.5 for 1.5%): 1.37
Enter number of shares for VTI: 10
Enter price per share on purchase date for VTI ($): 206.55
Enter purchase date for VTI (YYYY-MM-DD): 2022-05-31
Enter current/analysis date for VTI (YYYY-MM-DD): 2025-05-19
Enter current/analysis price per share for VTI ($): 292.81

Enter details for ETF #2:
------------------------------
Enter ETF #2 name (e.g., VTI, BND): BND
Enter expense ratio for BND (as percentage, e.g., 0.05 for 0.05%): 0.03
Enter current annual dividend yield for BND (as percentage, e.g., 1.5 for 1.5%): 3.68
Enter number of shares for BND: 5
Enter price per share on purchase date for BND ($): 76.55
Enter purchase date for BND (YYYY-MM-DD): 2022-05-31
Enter current/